# Snowpark Basics HoL Part 1 - DataFrame Basics

## 1.1 Setup

### Imports
These imports are from our local Python environment, snowparkbasics. Look out for F. and T. below.

In [1]:
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T

import sys
import json
import pandas as pd
import numpy as np

# Make sure we do not get line breaks when doing show on wide dataframes
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

### Create Snowpark Session
Using a credentials file simplifies the HoL but is not recommended as good practice for development or production environments.
<br> The Python connector documentation explains how to use other authentication methods.

In [6]:
with open('creds.json') as f:
    connection_parameters = json.load(f)

In [7]:
session = Session.builder.configs(connection_parameters).create()
print(f"Current Database and schema: {session.get_fully_qualified_current_schema()}")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Database and schema: "FROSTBYTE_TASTY_BYTES_V2"."RAW_POS"
Current Warehouse: "TASTY_DE_WH"


### Modifying our Session
We can use **session.sql** to issue any 'SQL' command. Note that due to lazy evaluation, typically nothing will happen without a show() or collect().

In [8]:
session.sql("USE WAREHOUSE TASTY_DEV_WH").collect()
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Warehouse: "TASTY_DEV_WH"


However, session also has a number of methods such as **use_warehouse()**. These *are* run immediately.

In [9]:
session.use_warehouse("TASTY_DE_WH")
print(f"Current Warehouse: {session.get_current_warehouse()}")

Current Warehouse: "TASTY_DE_WH"


## 1.2 Loading a DataFrame

### Pandas DataFrames from CSV
Let's create a Pandas dataframe directly from csv.

In [10]:
# Creating a Pandas DataFrame - the order header csv is in fact the data from only one truck!!
pandas_truck_df = pd.read_csv('data/truck.csv')
pandas_header_df = pd.read_csv('data/header.csv')
print(type(pandas_truck_df))

<class 'pandas.core.frame.DataFrame'>


In [11]:
# Displaying the Pandas dataframe
pandas_truck_df

,TRUCK_ID,MENU_TYPE_ID,PRIMARY_CITY,REGION,ISO_REGION,COUNTRY,ISO_COUNTRY_CODE,FRANCHISE_FLAG,YEAR,MAKE,MODEL,EV_FLAG,FRANCHISE_ID,TRUCK_OPENING_DATE
0,2,2,San Mateo,California,CA,United States,US,0,2015,Ford_,Step Van,0,1,2015-07-01
1,3,3,San Mateo,California,CA,United States,US,1,2004,Freightliner,MT45 Utilimaster,0,2,2015-11-01
2,4,4,San Mateo,California,CA,United States,US,1,1997,Chevrolet,P30,1,3,2019-02-01
3,5,5,San Mateo,California,CA,United States,US,1,2010,Custom,Van,1,4,2020-04-01
4,6,6,San Mateo,California,CA,United States,US,1,2010,Airstream,Trailer,0,5,2015-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,392,2,Madrid,Madrid provincia,Madrid,Spain,ES,0,2010,Ford_,Step Van,0,1,2014-12-29
446,396,6,Madrid,Madrid provincia,Madrid,Spain,ES,1,2005,Freightliner,MT45 Utilimaster,1,298,2014-12-29
447,419,14,Barcelona,Barcelona provincia,Barcelona,Spain,ES,1,2015,Airstream,Trailer,0,312,2014-12-29
448,427,7,Cairo,Cairo Governorate,Al Qahirah,Egypt,EG,1,2005,Ford_,Step Van,0,319,2014-12-29


### Snowpark DataFrames from Tables

The Snowpark **Table** class is a child of the **DataFrame** class.  We can define a dataframe based on a table very simply.
<br> (We'll look at loading file data into tables in Part 3.)

In [12]:
snowpark_truck_df = session.table('TRUCK')
snowpark_header_df = session.table('ORDER_HEADER')
print(type(snowpark_truck_df))

<class 'snowflake.snowpark.table.Table'>


### Comparing DataFrames
Compare sizes

In [13]:
print(
    "Size in MB of Pandas Truck DataFrame in Memory: ",
    np.round(sys.getsizeof(pandas_truck_df) / (1024.0**2), 2),
)
print(
    "Size in MB of Snowpark Truck DataFrame in Memory: ",
    np.round(sys.getsizeof(snowpark_truck_df) / (1024.0**2), 2),
)
print(
    "Size in MB of Pandas Header DataFrame in Memory: ",
    np.round(sys.getsizeof(pandas_header_df) / (1024.0**2), 2),
)
print(
    "Size in MB of Snowpark Header DataFrame in Memory: ",
    np.round(sys.getsizeof(snowpark_header_df) / (1024.0**2), 2),
)

Size in MB of Pandas Truck DataFrame in Memory:  0.25
Size in MB of Snowpark Truck DataFrame in Memory:  0.0
Size in MB of Pandas Header DataFrame in Memory:  132.66
Size in MB of Snowpark Header DataFrame in Memory:  0.0


The only thing stored in a Snowpark DataFrame is the SQL needed to return data.
<br>Trying to manipulate even one truck's worth of order headers in Pandas starts to get 'interesting'.

Now, what is going on under the covers? You might want to log into your Snowflake account as the same user and review Snowsight Query History. But you can also use this DataFrame attribute from Snowpark...

In [14]:
snowpark_header_df.queries

{'queries': ['SELECT  *  FROM (ORDER_HEADER)'], 'post_actions': []}

A Snowpark DataFrame can be converted to a Pandas DataFrame. This will pull the data from Snowflake into local memory.

In [15]:
pandas_truck_df2 = snowpark_truck_df.to_pandas()
pandas_truck_df2

,TRUCK_ID,MENU_TYPE_ID,PRIMARY_CITY,REGION,ISO_REGION,COUNTRY,ISO_COUNTRY_CODE,FRANCHISE_FLAG,YEAR,MAKE,MODEL,EV_FLAG,FRANCHISE_ID,TRUCK_OPENING_DATE
0,2,2,San Mateo,California,CA,United States,US,0,2015,Ford_,Step Van,0,1,2015-07-01
1,3,3,San Mateo,California,CA,United States,US,1,2004,Freightliner,MT45 Utilimaster,0,2,2015-11-01
2,4,4,San Mateo,California,CA,United States,US,1,1997,Chevrolet,P30,1,3,2019-02-01
3,5,5,San Mateo,California,CA,United States,US,1,2010,Custom,Van,1,4,2020-04-01
4,6,6,San Mateo,California,CA,United States,US,1,2010,Airstream,Trailer,0,5,2015-07-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,392,2,Madrid,Madrid provincia,Madrid,Spain,ES,0,2010,Ford_,Step Van,0,1,2014-12-29
446,396,6,Madrid,Madrid provincia,Madrid,Spain,ES,1,2005,Freightliner,MT45 Utilimaster,1,298,2014-12-29
447,419,14,Barcelona,Barcelona provincia,Barcelona,Spain,ES,1,2015,Airstream,Trailer,0,312,2014-12-29
448,427,7,Cairo,Cairo Governorate,Al Qahirah,Egypt,EG,1,2005,Ford_,Step Van,0,319,2014-12-29


Both our Pandas DataFrames have the same shape

In [16]:
pandas_truck_df.shape, pandas_truck_df2.shape

((450, 14), (450, 14))

### Displaying a Snowpark DataFrame
Defining and modifying a Snowpark dataframe does not generally result in any activity within Snowflake - lazy evaluation, similar to Spark.
The **show** method causes a query to be generated and sent and data returned - by default just 10 rows.
In contrast **toPandas** or **to_pandas** will retrieve the whole dataset unless you set a **limit**. 

In [17]:
snowpark_header_df.show() # <- has a default limit of 10, and prints the data out
snowpark_header_df.limit(5).toPandas() # <- collects first 5 rows and displays as pandas-dataframe

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"CUSTOMER_ID"  |"DISCOUNT_ID"  |"SHIFT_ID"  |"SHIFT_START_TIME"  |"SHIFT_END_TIME"  |"ORDER_CHANNEL"  |"ORDER_TS"                  |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"  |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"  |
------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|32723511    |140         |3193.0         |NULL           |NULL           |1           |09:00:00            |15:00:00          |NULL 

,ORDER_ID,TRUCK_ID,LOCATION_ID,CUSTOMER_ID,DISCOUNT_ID,SHIFT_ID,SHIFT_START_TIME,SHIFT_END_TIME,ORDER_CHANNEL,ORDER_TS,SERVED_TS,ORDER_CURRENCY,ORDER_AMOUNT,ORDER_TAX_AMOUNT,ORDER_DISCOUNT_AMOUNT,ORDER_TOTAL
0,32723511,140,3193.0,NaN,None,1,09:00:00,15:00:00,None,2017-08-15 11:00:00.355,None,USD,16.36,None,None,16.36
1,32723512,224,6654.0,NaN,None,1,09:00:00,15:00:00,None,2017-08-15 13:11:52.893,None,USD,7.93,None,None,7.93
2,32723513,415,13900.0,NaN,None,1,09:00:00,15:00:00,None,2017-08-15 10:47:57.132,None,USD,2.60,None,None,2.60
3,32723514,300,7837.0,NaN,None,1,09:00:00,15:00:00,None,2017-08-15 14:23:58.355,None,USD,3.17,None,None,3.17
4,32723515,216,13815.0,NaN,None,1,09:00:00,15:00:00,None,2017-08-15 09:51:18.599,None,USD,1.44,None,None,1.44


In fact, you don't need to give your dataframe a name just to examine the table.

In [18]:
session.table("RAW_POS.LOCATION").show()

----------------------------------------------------------------------------------------------------------------------------------------
|"LOCATION_ID"  |"PLACEKEY"           |"LOCATION"                                |"CITY"   |"REGION"  |"ISO_COUNTRY_CODE"  |"COUNTRY"  |
----------------------------------------------------------------------------------------------------------------------------------------
|5048           |zzy-222@53v-7xj-nqz  |L'Enclos des Oiseaux                      |Paris    |Paris     |FR                  |France     |
|9612           |zzy-224@53v-7y8-jn5  |Esplanade Jacques Chaban Delmas           |Paris    |Paris     |FR                  |France     |
|10386          |zzy-225@53v-8dm-bff  |College Boris Vian                        |Paris    |Paris     |FR                  |France     |
|10875          |zzy-226@53v-7yb-3dv  |Refectoire des Cordeliers                 |Paris    |Paris     |FR                  |France     |
|12260          |zzy-22c@53v-7yb-47q  |Sq

### Simple DataFrame Information

The **count** method on a DataFrame will return the number of rows. This also triggers a query to Snowflake. *(Cf pyspark.sql.DataFrame.count())*

In [19]:
# Number of rows in dataset
snowpark_header_df.count()

84240060

We can get an idea of the structure from the **schema** attribute.  *(Cf pyspark.sql.DataFrame.schema)*

In [20]:
header_schema = snowpark_header_df.schema
header_schema

StructType([StructField('ORDER_ID', LongType(), nullable=True), StructField('TRUCK_ID', LongType(), nullable=True), StructField('LOCATION_ID', DoubleType(), nullable=True), StructField('CUSTOMER_ID', LongType(), nullable=True), StructField('DISCOUNT_ID', StringType(16777216), nullable=True), StructField('SHIFT_ID', LongType(), nullable=True), StructField('SHIFT_START_TIME', TimeType(), nullable=True), StructField('SHIFT_END_TIME', TimeType(), nullable=True), StructField('ORDER_CHANNEL', StringType(16777216), nullable=True), StructField('ORDER_TS', TimestampType(tz=ntz), nullable=True), StructField('SERVED_TS', StringType(16777216), nullable=True), StructField('ORDER_CURRENCY', StringType(3), nullable=True), StructField('ORDER_AMOUNT', DecimalType(38, 4), nullable=True), StructField('ORDER_TAX_AMOUNT', StringType(16777216), nullable=True), StructField('ORDER_DISCOUNT_AMOUNT', StringType(16777216), nullable=True), StructField('ORDER_TOTAL', DecimalType(38, 4), nullable=True)])

Using the **describe** method will return some basic statistics for all numeric and string columns.  *(Cf pyspark.sql.DataFrame.describe())*
<br>Note that this does real work inside Snowflake! The statistical values are not necessarily meaningful for all columns.
<br>Can you find the maximum order value in the data? 

In [21]:
# Calculating various statistics per column
snowpark_header_df.describe().show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SUMMARY"  |"ORDER_ID"         |"TRUCK_ID"          |"LOCATION_ID"      |"CUSTOMER_ID"      |"DISCOUNT_ID"  |"SHIFT_ID"          |"ORDER_CHANNEL"  |"SERVED_TS"  |"ORDER_CURRENCY"  |"ORDER_AMOUNT"      |"ORDER_TAX_AMOUNT"  |"ORDER_DISCOUNT_AMOUNT"  |"ORDER_TOTAL"       |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|count      |84240060.0         |84240060.0          |84240060.0         |7174101.0          |0              |84240060.0          |0                |0            |84240060          |84

## 1.3 Managing Columns

### Selecting Columns
There are several ways to **select** specific columns, including **functions.col** and **DataFrame.col**. 
<br>The latter two are needed in several stuations to avoid ambiguities with string constants. 
<br>What do you notice about the four results below?

In [22]:
header_df1 = snowpark_header_df.select('ORDER_ID','TRUCK_ID','LOCATION_ID','ORDER_AMOUNT','ORDER_TS')
header_df2 = snowpark_header_df[['ORDER_ID','TRUCK_ID','LOCATION_ID','ORDER_AMOUNT','ORDER_TS']] # -> pandas-like selection
header_df3 = snowpark_header_df.select(F.col("ORDER_ID"),F.col("truck_id"),F.col("location_id"),
                                       F.col("order_amount"), F.col("order_ts"))
header_df4 = snowpark_header_df.select(snowpark_header_df.col('ORDER_ID'),snowpark_header_df.col('TRUCK_ID'),
                                       snowpark_header_df.col('LOCATION_ID'),snowpark_header_df.col('ORDER_AMOUNT'),
                                       snowpark_header_df.col('ORDER_TS'))
header_df1.show()
header_df2.show()
header_df3.show()
header_df4.show()

-----------------------------------------------------------------------------------------
|"ORDER_ID"  |"TRUCK_ID"  |"LOCATION_ID"  |"ORDER_AMOUNT"  |"ORDER_TS"                  |
-----------------------------------------------------------------------------------------
|32723511    |140         |3193.0         |16.3600         |2017-08-15 11:00:00.355000  |
|32723512    |224         |6654.0         |7.9300          |2017-08-15 13:11:52.893000  |
|32723513    |415         |13900.0        |2.6000          |2017-08-15 10:47:57.132000  |
|32723514    |300         |7837.0         |3.1700          |2017-08-15 14:23:58.355000  |
|32723515    |216         |13815.0        |1.4400          |2017-08-15 09:51:18.599000  |
|32723516    |61          |1280.0         |1.8000          |2017-08-15 10:17:15.885000  |
|32723517    |298         |10101.0        |43.6700         |2017-08-15 14:40:59.997000  |
|32723518    |256         |14472.0        |20.1200         |2017-08-15 14:20:01.203000  |
|32723519 

In general in Python single and double quotes are interchangeable. Best practice is to choose one and stick with it (unlike in this HoL). 
<br>Note that in all the examples above, the names are implicitly converted to uppercase.
<br>To handle identifiers with lowercase you need to add explicit double quotes within the string, either within single quotes as below, or with an escape character. 

In [23]:
#The following statement should fail
snowpark_header_df.select(F.col('"order_id"'),F.col("TRUCK_ID"),F.col("LOCATION_ID"),F.col("ORDER_AMOUNT"), F.col("ORDER_TS")).show()

SnowparkSQLException: (1304): 01b0a210-0404-d76a-001b-7a8701edc7d6: 000904 (42000): SQL compilation error: error line 1 at position 7
invalid identifier '"order_id"'

### Casting, Aliasing and In-Line Calculations
We can **cast** the column datatypes. For example ORDER_AMOUNT could be cast to NUMBER(36,2). Alternatively we have functions like **to_date**.

In [24]:
header_df1 = snowpark_header_df.select(F.col("ORDER_ID"),F.col('ORDER_AMOUNT').cast(T.DecimalType(36,2)),
                                       F.to_date(F.col('ORDER_TS')))
header_df1.show()

-------------------------------------------------------------------------------------
|"ORDER_ID"  |"CAST (""ORDER_AMOUNT"" AS NUMBER(36, 2))"  |"TO_DATE(""ORDER_TS"")"  |
-------------------------------------------------------------------------------------
|120684509   |17.58                                       |2022-09-08               |
|120684510   |16.88                                       |2022-09-08               |
|120684511   |9.68                                        |2022-09-08               |
|120684512   |64.05                                       |2022-09-08               |
|120684513   |10.20                                       |2022-09-08               |
|120684514   |13.12                                       |2022-09-08               |
|120684515   |12.03                                       |2022-09-08               |
|120684516   |16.22                                       |2022-09-08               |
|120684517   |37.66                                   

That's a bit ugly. Let's alias those columns...  **alias**, **name** and **as_** all achieve the same effect. *(Cf pyspark alias or name)*

In [25]:
header_df1 = snowpark_header_df.select(F.col("ORDER_ID"),F.col('ORDER_AMOUNT').cast(T.DecimalType(36,2)).alias("ORDER_AMOUNT_2D"),
                                      F.to_date(F.col('ORDER_TS')).alias('ORDER_DATE'))
header_df1.show()
header_df1.queries

-------------------------------------------------
|"ORDER_ID"  |"ORDER_AMOUNT_2D"  |"ORDER_DATE"  |
-------------------------------------------------
|142337652   |11.53              |2021-12-28    |
|142337653   |22.06              |2021-12-28    |
|142337654   |3.00               |2021-12-28    |
|142337655   |41.35              |2021-12-28    |
|142337656   |12.48              |2021-12-28    |
|142337657   |10.79              |2021-12-28    |
|142337658   |11.41              |2021-12-28    |
|142337659   |7.07               |2021-12-28    |
|142337660   |11.98              |2021-12-28    |
|142337661   |7.57               |2021-12-28    |
-------------------------------------------------



{'queries': ['SELECT "ORDER_ID",  CAST ("ORDER_AMOUNT" AS NUMBER(36, 2)) AS "ORDER_AMOUNT_2D", to_date("ORDER_TS") AS "ORDER_DATE" FROM ORDER_HEADER'],
 'post_actions': []}

We can also include calculated expressions within a select as we can in SQL. For example we can use + - * / ** arithmetic operators.

In [26]:
header_df2 = header_df1.select(F.col("ORDER_ID"),F.col('ORDER_AMOUNT_2D'),
                              (F.col('ORDER_AMOUNT_2D')*100).alias("OA_CENTS"))
header_df2.show()

-----------------------------------------------
|"ORDER_ID"  |"ORDER_AMOUNT_2D"  |"OA_CENTS"  |
-----------------------------------------------
|99331505    |32.62              |3262.00     |
|99331506    |20.89              |2089.00     |
|99331507    |18.72              |1872.00     |
|99331508    |26.55              |2655.00     |
|99331509    |3.91               |391.00      |
|99331510    |9.69               |969.00      |
|99331511    |23.08              |2308.00     |
|99331512    |4.81               |481.00      |
|99331513    |5.65               |565.00      |
|99331514    |47.84              |4784.00     |
-----------------------------------------------



Again, switch to Snowsight Query History and see what is going on...

### Adding and Removing Columns


To add a new calculated column to a Snowpark DataFrame the **withColumn** or **with_column** method can be used.  *(Cf pysaprk withColumn)*
In this example we are adding a new TRUCK column, AGE, that calculates the number of years since the YEAR. 
Note the use of F.col here - otherwise 'YEAR' could be seen as a string value. One approach is to use built-in Python functions to derive the current year locally.

In [27]:
import datetime
from datetime import datetime
year = datetime.now().year

truck_df1 = snowpark_truck_df.select('TRUCK_ID','REGION','ISO_COUNTRY_CODE','YEAR',
                                     'MAKE','MODEL','TRUCK_OPENING_DATE')
truck_df1 = truck_df1.withColumn('AGE', year - F.col('YEAR'))
truck_df1.to_pandas()

,TRUCK_ID,REGION,ISO_COUNTRY_CODE,YEAR,MAKE,MODEL,TRUCK_OPENING_DATE,AGE
0,2,California,US,2015,Ford_,Step Van,2015-07-01,8
1,3,California,US,2004,Freightliner,MT45 Utilimaster,2015-11-01,19
2,4,California,US,1997,Chevrolet,P30,2019-02-01,26
3,5,California,US,2010,Custom,Van,2020-04-01,13
4,6,California,US,2010,Airstream,Trailer,2015-07-01,13
...,...,...,...,...,...,...,...,...
445,392,Madrid provincia,ES,2010,Ford_,Step Van,2014-12-29,13
446,396,Madrid provincia,ES,2005,Freightliner,MT45 Utilimaster,2014-12-29,18
447,419,Barcelona provincia,ES,2015,Airstream,Trailer,2014-12-29,8
448,427,Cairo Governorate,EG,2005,Ford_,Step Van,2014-12-29,18


The following version pushes the current year 'calculation' down to Snowflake.

In this section we show how each new version of the dataframe can replace the previous one by using the same name.
This can make sense whilst we build up the dataframe query we really want. However, when we do this across cells and try to rerun just one cell we can get errors if a later statement has altered the structure that an earlier statement relied on.... We avoid that here by redefining truck_df1 from its source.

In [28]:
truck_df1 = snowpark_truck_df.select('TRUCK_ID','REGION','ISO_COUNTRY_CODE','YEAR',
                                     'MAKE','MODEL','TRUCK_OPENING_DATE')
truck_df1 = truck_df1.withColumn('AGE', F.date_part("year", F.current_date()) - F.col('YEAR'))
truck_df1.to_pandas()

,TRUCK_ID,REGION,ISO_COUNTRY_CODE,YEAR,MAKE,MODEL,TRUCK_OPENING_DATE,AGE
0,2,California,US,2015,Ford_,Step Van,2015-07-01,8
1,3,California,US,2004,Freightliner,MT45 Utilimaster,2015-11-01,19
2,4,California,US,1997,Chevrolet,P30,2019-02-01,26
3,5,California,US,2010,Custom,Van,2020-04-01,13
4,6,California,US,2010,Airstream,Trailer,2015-07-01,13
...,...,...,...,...,...,...,...,...
445,392,Madrid provincia,ES,2010,Ford_,Step Van,2014-12-29,13
446,396,Madrid provincia,ES,2005,Freightliner,MT45 Utilimaster,2014-12-29,18
447,419,Barcelona provincia,ES,2015,Airstream,Trailer,2014-12-29,8
448,427,Cairo Governorate,EG,2005,Ford_,Step Van,2014-12-29,18


If we do not want to use specific columns we can use **drop** to remove those from a Snowpark DataFrame.  
**Note:** This is not removing them from the underlying table.

In [29]:
# Drop a column
truck_df1 = truck_df1.drop('MODEL','YEAR')
truck_df1.show()

----------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"REGION"    |"ISO_COUNTRY_CODE"  |"MAKE"        |"TRUCK_OPENING_DATE"  |"AGE"  |
----------------------------------------------------------------------------------------------
|2           |California  |US                  |Ford_         |2015-07-01            |8      |
|3           |California  |US                  |Freightliner  |2015-11-01            |19     |
|4           |California  |US                  |Chevrolet     |2019-02-01            |26     |
|5           |California  |US                  |Custom        |2020-04-01            |13     |
|6           |California  |US                  |Airstream     |2015-07-01            |13     |
|7           |California  |US                  |Custom        |2019-02-01            |13     |
|8           |California  |US                  |Chevrolet     |2023-03-01            |24     |
|9           |California  |US                  |Cu

## 1.4 Simple Data Manipulation

### Filtering Rows
To filter/select specific rows we use **filter**.
A whole set of column operators are available to be used e.g. 

==, !=, <, <=, >, >=  for comparisons;   &, |  and or;  + - * / **  arithmetic operators

In [30]:
# Filter data
truck_df2 = truck_df1.filter(F.col('ISO_COUNTRY_CODE') == 'GB')
truck_df2.show()
truck_df3 = truck_df1.filter(F.col('ISO_COUNTRY_CODE').in_('ES','FR','GB')).sort('ISO_COUNTRY_CODE')
truck_df3.show()
truck_df4 = truck_df1.filter(F.col('ISO_COUNTRY_CODE').like('F%'))
truck_df4.show()
truck_df3.queries

--------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"REGION"        |"ISO_COUNTRY_CODE"  |"MAKE"        |"TRUCK_OPENING_DATE"  |"AGE"  |
--------------------------------------------------------------------------------------------------
|121         |Greater London  |GB                  |Ford_         |2017-04-01            |9      |
|122         |Greater London  |GB                  |Ford_         |2015-10-01            |16     |
|123         |Greater London  |GB                  |Ford_         |2018-01-01            |9      |
|124         |Greater London  |GB                  |Freightliner  |2022-01-01            |22     |
|125         |Greater London  |GB                  |Ford_         |2015-10-01            |9      |
|126         |Greater London  |GB                  |Ford_         |2018-02-01            |16     |
|127         |Greater London  |GB                  |Ford_         |2015-08-01            |9      |
|128      

{'queries': ['SELECT "TRUCK_ID", "REGION", "ISO_COUNTRY_CODE", "MAKE", "TRUCK_OPENING_DATE", (date_part(\'year\', current_date()) - "YEAR") AS "AGE" FROM TRUCK WHERE "ISO_COUNTRY_CODE" IN (\'ES\', \'FR\', \'GB\') ORDER BY "ISO_COUNTRY_CODE" ASC NULLS FIRST'],
 'post_actions': []}

### Sorting
We may want to see data in a specific order. For this the **sort** method is used...

In [31]:
# Sort data
truck_df4 = truck_df4.sort(F.col('TRUCK_OPENING_DATE').desc(),F.col('ISO_COUNTRY_CODE'))
truck_df4.show(20)

--------------------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"REGION"                    |"ISO_COUNTRY_CODE"  |"MAKE"        |"TRUCK_OPENING_DATE"  |"AGE"  |
--------------------------------------------------------------------------------------------------------------
|163         |Île-de-France               |FR                  |Freightliner  |2023-01-01            |18     |
|165         |Île-de-France               |FR                  |Freightliner  |2023-01-01            |22     |
|168         |Provence–Alpes–Côte-d'Azur  |FR                  |Chevrolet     |2023-01-01            |24     |
|155         |Île-de-France               |FR                  |Freightliner  |2023-01-01            |18     |
|176         |Provence–Alpes–Côte-d'Azur  |FR                  |Freightliner  |2021-10-01            |22     |
|158         |Île-de-France               |FR                  |Freightliner  |2020-07-01            |18     |
|

### Aggregation
To aggregate data the **groupBy** or **group_by** method is typically used. The groupby method produces a RelationalGroupedDataFrame object with its own specific methods, which, in turn, return a DataFrame. The **agg** method provides the most flexibility for managing the output, and including different aggregate metrics for different columns. Note the syntax - although operating on columns, the functions like avg expect the string of the column name.

In [32]:
truck_df5 = truck_df3.groupBy(['ISO_COUNTRY_CODE','MAKE']).agg(
             [F.count('*').alias('COUNT'),F.avg('AGE').alias('AVG_TRUCK_AGE'),F.max('TRUCK_ID').alias('MAX_TRUCK_ID')])
truck_df5 = truck_df5.sort(F.col('ISO_COUNTRY_CODE'), F.col('COUNT').desc())
truck_df5.queries

{'queries': ['SELECT  *  FROM ( SELECT "ISO_COUNTRY_CODE", "MAKE", count(1) AS "COUNT", avg("AGE") AS "AVG_TRUCK_AGE", max("TRUCK_ID") AS "MAX_TRUCK_ID" FROM ( SELECT "TRUCK_ID", "REGION", "ISO_COUNTRY_CODE", "MAKE", "TRUCK_OPENING_DATE", (date_part(\'year\', current_date()) - "YEAR") AS "AGE" FROM TRUCK WHERE "ISO_COUNTRY_CODE" IN (\'ES\', \'FR\', \'GB\') ORDER BY "ISO_COUNTRY_CODE" ASC NULLS FIRST) GROUP BY "ISO_COUNTRY_CODE", "MAKE") ORDER BY "ISO_COUNTRY_CODE" ASC NULLS FIRST, "COUNT" DESC NULLS LAST'],
 'post_actions': []}

In [33]:
truck_df5.show(15)

----------------------------------------------------------------------------------
|"ISO_COUNTRY_CODE"  |"MAKE"        |"COUNT"  |"AVG_TRUCK_AGE"  |"MAX_TRUCK_ID"  |
----------------------------------------------------------------------------------
|ES                  |Freightliner  |12       |19.416667        |417             |
|ES                  |Ford_         |8        |12.750000        |418             |
|ES                  |Airstream     |2        |10.500000        |419             |
|ES                  |Custom        |2        |14.000000        |420             |
|ES                  |Chevrolet     |2        |26.000000        |404             |
|ES                  |Citroën       |2        |43.500000        |415             |
|ES                  |Volkswagen    |1        |51.000000        |403             |
|ES                  |Nissan        |1        |5.000000         |414             |
|FR                  |Freightliner  |13       |19.538462        |178             |
|FR 

### Using SQL
How might we express the same combined query in SQL? It is quite likely that we would want to break it down in a similar way.
We can run SQL queries directly using **session.sql** (including Snowflake commands issued as SQL). 
Note that nothing will happen without a collect() or show().

In the example below, the three quotes beginning and end are how we indicate a multi-line string in Python.

In [34]:
truck_df6 = session.sql("""
 SELECT ISO_COUNTRY_CODE, MAKE, count(1) AS COUNT, avg(AGE) AS AVG_TRUCK_AGE, max(TRUCK_ID) AS MAX_TRUCK_ID 
   FROM ( SELECT TRUCK_ID, REGION, ISO_COUNTRY_CODE, MAKE, TRUCK_OPENING_DATE, (date_part('year', current_date()) - YEAR) AS AGE 
       FROM TRUCK WHERE ISO_COUNTRY_CODE IN ('ES', 'FR', 'GB')
     ) 
   GROUP BY ISO_COUNTRY_CODE, MAKE ORDER BY ISO_COUNTRY_CODE ASC NULLS FIRST, COUNT DESC NULLS LAST LIMIT 15
   """)
truck_df6.show()
truck_df6.queries

----------------------------------------------------------------------------------
|"ISO_COUNTRY_CODE"  |"MAKE"        |"COUNT"  |"AVG_TRUCK_AGE"  |"MAX_TRUCK_ID"  |
----------------------------------------------------------------------------------
|ES                  |Freightliner  |12       |19.416667        |417             |
|ES                  |Ford_         |8        |12.750000        |418             |
|ES                  |Citroën       |2        |43.500000        |415             |
|ES                  |Custom        |2        |14.000000        |420             |
|ES                  |Airstream     |2        |10.500000        |419             |
|ES                  |Chevrolet     |2        |26.000000        |404             |
|ES                  |Volkswagen    |1        |51.000000        |403             |
|ES                  |Nissan        |1        |5.000000         |414             |
|FR                  |Freightliner  |13       |19.538462        |178             |
|FR 

{'queries': ["SELECT ISO_COUNTRY_CODE, MAKE, count(1) AS COUNT, avg(AGE) AS AVG_TRUCK_AGE, max(TRUCK_ID) AS MAX_TRUCK_ID \n   FROM ( SELECT TRUCK_ID, REGION, ISO_COUNTRY_CODE, MAKE, TRUCK_OPENING_DATE, (date_part('year', current_date()) - YEAR) AS AGE \n       FROM TRUCK WHERE ISO_COUNTRY_CODE IN ('ES', 'FR', 'GB')\n     ) \n   GROUP BY ISO_COUNTRY_CODE, MAKE ORDER BY ISO_COUNTRY_CODE ASC NULLS FIRST, COUNT DESC NULLS LAST LIMIT 15"],
 'post_actions': []}

## 1.5 Persist Transformations

If we want to save the changes we can either save it as a table, meaning the SQL generated by the DataFrame is executed and the result is stored in a table or as a view where the DataFrame SQL will be the definition of the view.  
**save_as_table** saves the result in a table, if **mode='overwrite'** then it will also replace the data that is in it.

In [35]:
truck_df3.write.save_as_table(table_name='TRUCK_ANALYSIS', mode='overwrite')
session.table('TRUCK_ANALYSIS').show()

-------------------------------------------------------------------------------------------------------
|"TRUCK_ID"  |"REGION"             |"ISO_COUNTRY_CODE"  |"MAKE"        |"TRUCK_OPENING_DATE"  |"AGE"  |
-------------------------------------------------------------------------------------------------------
|419         |Barcelona provincia  |ES                  |Airstream     |2014-12-29            |8      |
|391         |Madrid provincia     |ES                  |Chevrolet     |2023-01-01            |26     |
|393         |Madrid provincia     |ES                  |Freightliner  |2021-10-01            |22     |
|394         |Madrid provincia     |ES                  |Freightliner  |2015-10-01            |23     |
|395         |Madrid provincia     |ES                  |Citroën       |2023-01-01            |45     |
|397         |Madrid provincia     |ES                  |Freightliner  |2015-08-01            |18     |
|398         |Madrid provincia     |ES                  |Freight

## 1.X YOUR TURN!

Here is the challenge: Generate a list of months for which we have data, the total order amount for each month (assume amounts are all held in the same currency), and the number of distinct locations visted in each month.
<br>Hints:
Functions you may find useful include **count_distinct** (aka countDistinct), **date_part**, **to_char** with numeric formatting '09' or 'FM09' and **concat**.

### Hint:   
To see all methods available use the TAB key.    F.<TAB>   Will show all functon methods.

To see for a specific function the help text.   SHIFT-TAB
                                                                                                                                                      
https://docs.snowflake.com/developer-guide/snowpark/reference/scala/com/snowflake/snowpark/index.html

### Check out the data

What columns in Order Header will you need?  

### Select the columns you need and create a Month column
You'll need to concatenate the year and month date parts of the order timestamp.
Note that (currently) date_format in Snowpark is an alias of to_date and not the date to string functionality of Pyspark...

### Now aggregate by month


In [ ]:
session.close()